## Huffman Encoding and Decoding Algorithm
## Copyright: Jagadeesh Vasudevamurthy
## filename:Huffman.ipynb

# All import here

In [1]:
import sys # For getting Python Version
import random 
import math
import heapq
import collections

# Node Class

In [2]:
class Node():
    def __init__(self, freq, data, l = None, r = None):
        # Nothing can be added in class Node
        # Must use Node class.Hacker rank fails without this Node
        self.freq= freq #Note public
        self.data=data #Note public
        self.left = l #Note public
        self.right = r #Note public
    ## YOU CAN ADD ANY NUMBER OF PRIVATE FUNCTIONS ONLY
    
    ##Override __lt__ 
    def __lt__(self,rhs:'Node')->'bool':
        if (self.freq < rhs.freq):  # We are building minheap
            return True
        return False

class MinHeap:
    def __init__(self):
        self._q = []
            
    def add(self,n:'Node'):
            heapq.heappush(self._q,n)
            
    def get_top(self)->'Node':
        return self._q[0]
    
    def get_top_and_remove(self)->'Node':
        n = heapq.heappop(self._q)
        return n  

# Huffman  Class

In [3]:
class Huffman():
    def __init__(self,show = False,f = None):
        ##Change this to the place where you write dot file
        self.base = "/Users/yashjain/Desktop/" 
        #Nothing can be changed or added below
        self._s = None
        self._show = show
        if (f):
            self._f = self.base + f
        self._id = []
        self.valueStore = {}
        ##YOU CAN ADD YOUR PRIVATE VARIABLE HERE
        
    #############################################
    #  All public functions below
    #  NOTHING CAN BE CHANGED BELOW
    #############################################
    def encode(self, s:'str')->'str':
        self._s = s
        return self._encode()

    def decode(self,e:'str')->'str':
        return self._decode(e)


    #############################################
    #  WRITE YOUR CODE BELOW
    #  All private functions below
    #############################################
    def _encode(self)->'str':
        return self.makeTree()

    def _decode(self, e:'str')->'str':
        return self.decodeString(e)
    
    def makeTree(self):
        print('--------------------------------- Step 1: Character occurrence ---------------------------------')
        self.characterOccur()
        self.leaveNodeBuild()
        print('--------------------------------- Step 6: Encoding ---------------------------------')
        data = ''
        for i in self._s:
            data+= self.valueStore[i]
        print("Original string:", self._s)
        print("Encoded string:", data)
        return data
    
    def leaveNodeBuild(self):
        counter = len(self._id._q)
        if len(self._id._q) > 1:
            print('--------------------------------- Step 3: Internal Node of the tree is Build in this order ---------------------------------')
            while len(self._id._q) != 1:
                top1 = self._id.get_top_and_remove()
                top2 = self._id.get_top_and_remove()
                newParentCount = top1.freq + top2.freq
                newParentName =  top1.data + top2.data
                counter+=1
                print('Internal Node', counter, ':' ,newParentName, newParentCount)
                newParent = Node(newParentCount, newParentName,top1, top2)
                self._id.add(newParent)
            print('--------------------------------- Step 4: Write dot file at location -----------------', self.base)
            self.generate_dot_file(self._id.get_top(), False)
            print('--------------------------------- Step 5: code for each character is as follows ---------------------------------')
            self.inorderTravser(self._id.get_top())
        else:
            print('--------------------------------- Step 4: Write dot file at location -----------------', self.base)
            self.generate_dot_file(self._id.get_top(), True)
            print('--------------------------------- Step 5: code for each character is as follows ---------------------------------')
            self.valueStore[self._id.get_top().data] = '0'
        print('Tree has', counter, 'nodes')
        
    def generate_dot_file(self, root, leaf):
        levelorder = []
        self.level_order_traverse(root, levelorder)
        dot = 'digraph g{\n\tlabel="'+self._s+'"\n'
        if(leaf):
            dot += '\t\t"' + root.data + '\\n' + str(root.freq) +'"\n'
        else:
            for i in range(len(levelorder)):
                for j in range(len(levelorder[i])):
                    node = levelorder[i][j]
                    if(node.left != None and node.right != None):
                        dot += '\t\t"' + node.data + '\\n' + str(node.freq) +'" -> "'+ node.left.data + '\\n' + str(node.left.freq) + '"[color=red]\n'
                        dot += '\t\t"' + node.data + '\\n' + str(node.freq) +'" -> "'+ node.right.data + '\\n' + str(node.right.freq) + '"[color=blue]\n'
        dot += '}'
        f = open(self._f, "w")
        f.write(dot)
        f.close()
    
    def level_order_traverse(self, root, ans):
        if root is None:
            return ans
        q = collections.deque()
        q.append(root)
        while(q):
            q_len = len(q)
            level_node_list = []
            while(q_len > 0):
                curr_node = q.popleft()
                level_node_list.append(curr_node)
                q_len -= 1
                if(curr_node.left is not None):
                    q.append(curr_node.left)
                if(curr_node.right is not None):
                    q.append(curr_node.right)
            ans.append(level_node_list)

            
    def characterOccur(self):
        data = {}
        for i in self._s:
            if i not in data:
                data[i] = 1
            else:
                data[i]+=1
        self._id = MinHeap()
        for i in data:
            print(i, 'ocuurs', data[i], 'times')
        print('--------------------------------- Step 2: Tree Leave is build in this order ---------------------------------')
        counter = 0
        for i in data:
            counter+=1
            print('Leaf Node', counter, ':' ,i, data[i])
            node = Node(data[i],i)
            self._id.add(node)

    def inorderTravser(self, node, value=''):
        if not node:
            return
        self.inorderTravser(node.left, value + '0')
        if len(node.data) == 1:
            print(node.data, 'Code is', value)
            self.valueStore[str(node.data)] = value
        self.inorderTravser(node.right, value + '1')
    
    def decodeString(self, e):
        counter = 0
        string = ''
        while counter < len(e):
            tree = self._id.get_top()
            if len(e) ==1:
                if tree.left == None and tree.right == None:
                    string+=tree.data
                    counter+=1
            else:
                while tree != None:
                    if tree.left == None and tree.right == None:
                        string+=tree.data
                        break
                    if e[counter] == '1':
                        tree = tree.right
                        counter+=1
                    else:
                        tree = tree.left
                        counter+=1
        print("Decoded string:", string)
        return string
            
        
############################################# 
##LEETCODE  
## https://leetcode.com/problems/encode-and-decode-tinyurl/
## Step1: Delete all code and comment in Leetcpde
## Step2: Cut and paste this file in Leetcode
## Step3: Make False to True
## All tests must pass in one shot.
## Post screen shot of Leetcode passed as pdf
#############################################
if (False):
    Codec = Huffman

# Util class
# Nothing can be changed
# No need to write any code here

In [4]:
############################################################
# Util.py
# Author: Jagadeesh Vasudevamurthy
# Copyright: Jagadeesh Vasudevamurthy 2020
###########################################################

############################################################
# NOTHING CAN BE CHANGED IN THIS FILE
###########################################################

############################################################
# All imports
###########################################################
import sys # For getting Python Version
import random 
import math
from time import process_time 

class Util():
  pass

  ############################################
  # generate_random_number start to end INCLUDED 
  # start to end INCLUDED
  #########################################
  def generate_a_random_number(self,start:int,end:int)->'int':
    v = random.randrange(start,end+1);
    return v

  ############################################
  # generate_random_number GENERATES  N random numbers betweem 
  # start to end INCLUDED
  # if onlypositive is False, generates both pos and negative number
  #  randrange(beg, end, step) :- 
  #  beginning number (included in generation), 
  #  last number (excluded in generation) a
  #  nd step ( to skip numbers in range while selecting).
  #########################################
  def generate_random_number(self, N:int, onlypositive:bool, start:int, end:int)->'List of integer':
    a = []
    for i in range(N):
      v = self.generate_a_random_number(start,end);
      if (onlypositive == False):
        if ((i % 2) == 0): ##//Even. Half are positive, Half are negative
          v = -v ;
      a.append(v)
    return a

  ############################################
  # swap
  #########################################
  def swap(self,a:'List of integer', i:'int', j:'int'):
    t = a[i]
    a[i] = a[j]
    a[j] = t

  ############################################
  # generate shuffled number between 0 to n
  # n-1 not encluded
  #########################################
  def generate_suffled_number_between_1_to_n(self, n:int)->'List of integer':
    a = []
    for i in range(n):
      a.append(i)

    for i in range(n):
      j = self.generate_a_random_number(0,n-1);
      k = self.generate_a_random_number(0,n-1);
      self.swap(a,j,k)
    return a

  ############################################
  # generate shuffled number between 0 to n
  # n-1 not encluded
  #########################################
  def generate_duplicated_number_between_1_to_n(self, n:int)->'List of integer':
    a = []
    for i in range(n):
      a.append(i)

    for i in range(n):
      j = self.generate_a_random_number(0,n-1);
      k = self.generate_a_random_number(0,n-1);
      a[k] = a[j]
    return a

  ############################################
  # generate n numbers in ascending order from 0 to n-1
  #########################################
  def generate_n_numbers_in_ascending_order(self, n:int)->'List of integer':
    a = []
    for i in range(n):
      a.append(i)
    return a

  ############################################
  # generate n numbers in descending order from n-1 to 0
  #########################################
  def generate_n_numbers_in_descending_order(self, n:int)->'List of integer':
    a = []
    for i in range(n-1,-1,-1):
      a.append(i)
    return a

  ############################################
  # generate n same k number
  #########################################
  def generate_n_same_k_number(self, n:int,k:'int')->'List of integer':
    a = []
    for i in range(n):
      a.append(k)
    return a

  ############################################
  # print_index(10)
  #    0   1   2   3   4   5   6   7   8   9
  #########################################
  def print_index(self, n:int):
    for i in range(n):
      print("{:4d}".format(i),end="")
    print()

  ############################################
  # a = [7,8,9, 23, 100]
  # print_list(a)
  # 7   8   9  23 100
  #########################################
  def print_list(self, a:'list'):
    for i in range(len(a)):
      print("{:4d}".format(a[i]),end="")
    print()

  ############################################
  # a = [7,8,9, 1, 100]
  # crash
  #########################################
  def assert_ascending_range(self, a:'list',start:int, includingend:int):
    t = a[start]
    for i in range(start+1,includingend):
      if (a[i] < t):
        assert(False)
      t = a[i]

  ############################################
  # a = [7,8,9, 1, 100]
  # crash
  #########################################
  def assert_ascending(self, a:'list'):
    if (len(a)):
      self.assert_ascending_range(a,0,len(a)) 

  ############################################
  # a = [1,2,3, 3, 4]
  # return True
  #########################################
  def is_ascending_order_has_duplicates(self,a:'list')->'bool':
    if (len(a)):
        t = a[0]
        for i in range(1,len(a)):
          if (a[i] <= t):
            return True
          t = a[i]
    return False

  ############################################
  # log to the next possible integer
  #########################################
  def log_upper_bound(self, n:'int', b:'int')->'int':
    f = math.log(n,b)
    c = math.ceil(f)
    return c 

  ############################################
  # log to the smallest possible integer
  #########################################
  def log_lower_bound(self, n:'int', b:'int')->'int':
    f = math.log(n,b)
    c = math.floor(f)
    return c 

  ############################################
  # sqrt to the next possible integer
  #########################################
  def sqrt_upper_bound(self, n:'int')->'int':
    f = math.sqrt(n)
    c = math.ceil(f)
    return c 

  ############################################
  # sqrt to the smallest possible integer
  #########################################
  def sqrt_lower_bound(self, n:'int')->'int':
    f = math.sqrt(n)
    c = math.floor(f)
    return c 
 

# Huffman test class
# NOTHING CAN BE CHANGED BELOW

In [5]:
############################################################
# HuffmanTest.py
# Author: Jagadeesh Vasudevamurthy
# Copyright: Jagadeesh Vasudevamurthy 2022
###########################################################

############################################################
#           NOTHING CAN BE CHANGED BELOW
###########################################################

############################################################
# All imports
# Comment all these imports in jupyter note book
###########################################################
##
##from Huffman import *
##from Util import *
##

class HuffmanTest():
    def __init__(self):
        #Nothing can be added here
        self.base = "C:\\scratch\\outputs\\huffmanpy\\"
        self.num = 1
        self._test_basic()
 
    ###########################################
    #           test1
    ############################################
    def _test1(self,s:'string',show:'bool',f:'string'):
        print("---- Problem",self.num,"s = ",s, "------  STARTS");
        h = Huffman(show,f)
        e = h.encode(s)
        d = h.decode(e)
        print("Original string:", s)
        print("Your     string:", d)
        if (s != d):
            assert(False)
        sl = len(s) * 8
        el = len(e)
        r = ((el - sl)/sl) * 100 ;
        print("% reduction =",r)

    ###########################################
    #           basic tests
    ############################################
    def _test_basic(self):
        show = True
        print("----------- Basic test Start --------------")
        self._test1("a",show,"1.dot");
        self._test1("aba",show,"2.dot");
        self._test1("aaabbggggghhhhaaaggggaaaaa_+@#",show,"3.dot");
        self._test1("A quick brown fox jumps over the lazy dog",show,"4.dot");
        self._test1("Pack my box with five dozen liquor jugs",show,"5.dot");
        self._test1("Long years ago we made a tryst with destiny, and now the time comes when we shall redeem our pledge, not wholly or in full measure, but very substantially.At the stroke of the midnight hour, when the world sleeps, India will awake to life and freedom. A moment comes, which comes but rarely in history, when we step out from the old to the new, when an age ends, and when the soul of a nation, long suppressed, finds utterance.",show,"6.dot");
        self._test1("Baa, baa, black sheep, have you any wool?",show,"7.dot");
        print("-----------Basic test Passed -------------------")


############################################################
# main 
# YOU CANNOT CHANGE ANYTHING BELOW
###########################################################
def main():
  print("Testing Huffman Starts")
  t = HuffmanTest()
  print("If you like my class, can uou please recommend on https://www.linkedin.com/in/jagadeesh-vasudevamurthy-6796591/")
  print("Testing Huffman Ends")

############################################################
# Real Main
###########################################################
if (__name__  == '__main__'):
  main()
  

Testing Huffman Starts
----------- Basic test Start --------------
---- Problem 1 s =  a ------  STARTS
--------------------------------- Step 1: Character occurrence ---------------------------------
a ocuurs 1 times
--------------------------------- Step 2: Tree Leave is build in this order ---------------------------------
Leaf Node 1 : a 1
--------------------------------- Step 4: Write dot file at location ----------------- /Users/yashjain/Desktop/
--------------------------------- Step 5: code for each character is as follows ---------------------------------
Tree has 1 nodes
--------------------------------- Step 6: Encoding ---------------------------------
Original string: a
Encoded string: 0
Decoded string: a
Original string: a
Your     string: a
% reduction = -87.5
---- Problem 1 s =  aba ------  STARTS
--------------------------------- Step 1: Character occurrence ---------------------------------
a ocuurs 2 times
b ocuurs 1 times
--------------------------------- Step 2: T